<a href="https://colab.research.google.com/github/seecode4/seeRepo1/blob/main/capstone/explore_triviaqa/trivia_qa_get_entries_ijson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Capstone Step5 - Explore triviaqa dataset

The dataset used here was download to a local GDrive from the [github reference](https://github.com/mandarjoshi90/triviaqa) for the paper "TriviaQA: A Large Scale Distantly Supervised Challenge Dataset for Reading Comprehension".  

A summary of the findings are at the end of this notebook.

In [1]:
# Mount GDrive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# https://github.com/mandarjoshi90/triviaqa
#   !npx degit fredzett/rmqa/data data
%cd /content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/see_trivia_qa_try1/
!pwd
!ls -lrt ./data_local/triviaqa-rc/qa
# !tar xfz ./triviaqa-rc.tar.gz

/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/see_trivia_qa_try1
/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/see_trivia_qa_try1
total 586437
-rw------- 1 root root  17318460 May  4  2017 wikipedia-dev.json
-rw------- 1 root root  31024171 May  4  2017 web-test-without-answers.json
-rw------- 1 root root    797321 May  4  2017 verified-wikipedia-dev.json
-rw------- 1 root root   4807932 May  4  2017 wikipedia-test-without-answers.json
-rw------- 1 root root 132411311 May  4  2017 wikipedia-train.json
-rw------- 1 root root   1096903 May  4  2017 verified-web-dev.json
-rw------- 1 root root 365397204 May  4  2017 web-train.json
-rw------- 1 root root  47655088 May  4  2017 web-dev.json


In [3]:
%%time
import os
import pandas as pd
!pip install ijson
import ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.9 MB/s eta 0:00:00
CPU times: user 497 ms, sys: 69.4 ms, total: 566 ms
Wall time: 5.76 s


In [4]:
%%time
%cd /content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/see_trivia_qa_try1/
dir_path = './data_local/triviaqa-rc/qa/'
# os.chdir(dir_path)
files = os.listdir(dir_path)
fsampcnt = {}
# Iterate through the list and print the name of each file
for file in files:
    # if os.path.isfile(os.path.join(directory_path, file)):
    if file.endswith(".json"):
      fname = dir_path+file
      num_lines = sum(1 for line in open(fname))
      fsampcnt[fname] = num_lines
for fname, scnt in fsampcnt.items():
  print(f'Num lines in {fname}: {scnt}')

/content/drive/MyDrive/GDcc4/ColabNotebooks/cc4_capstone/see_trivia_qa_try1
Num lines in ./data_local/triviaqa-rc/qa/wikipedia-dev.json: 450006
Num lines in ./data_local/triviaqa-rc/qa/verified-wikipedia-dev.json: 20553
Num lines in ./data_local/triviaqa-rc/qa/web-test-without-answers.json: 513040
Num lines in ./data_local/triviaqa-rc/qa/wikipedia-test-without-answers.json: 124646
Num lines in ./data_local/triviaqa-rc/qa/verified-web-dev.json: 26196
Num lines in ./data_local/triviaqa-rc/qa/wikipedia-train.json: 3445860
Num lines in ./data_local/triviaqa-rc/qa/web-train.json: 7118751
Num lines in ./data_local/triviaqa-rc/qa/web-dev.json: 928803
CPU times: user 4.91 s, sys: 426 ms, total: 5.34 s
Wall time: 15 s


In [ ]:
# import textwrap
# import pprint
# # pp = pprint.PrettyPrinter(indent=1)
# pp = pprint.PrettyPrinter(width=101, compact=True)
# print('Num of Samples:\n')
# pp.pprint(fsampcnt)

In [43]:
%%time
# dict_keys(['Answer', 'EntityPages', 'Question', 'QuestionId', 'QuestionSource', 'Searchone_qa_dict'])
# Data is a list of dict entries
def getqa_entries(kvitems, keys, numqa):
    """
    Get a list of numqa entries; each entry is a dict with keys and values
    if numqa < 1 get all entries
    """
    one_qa_dict = {} ; qa_dict_list = []
    qacnt = 1 ;
    kcnt = 0 ; klen = len(keys)
    for k, v in kvitems:
      # cnt += 1
      if k in keys:
        kcnt += 1
        one_qa_dict[k] = (v)
        # if 'Answer' not in one_qa_dict.keys():
        #   one_qa_dict['Answer'] = {'Value': 'Null'}
        if kcnt == klen:
          qa_dict_list.append(one_qa_dict)
          # if (qacnt < 5): # or (qacnt % 10000 == 0):
          #   print(one_qa_dict['Answer'])
          one_qa_dict = {}
          kcnt = 0
          qacnt += 1
      if (numqa > 0) and (qacnt > numqa):
        break
    return qa_dict_list

def getqa_entries_from_file(fname, numqa):
    """
    Get a list of numqa entries from given file
    Each entry is a dict with keys and values
    if numqa < 1 get all entries
    """
    qakeys = ['Answer', 'EntityPages', 'Question', 'QuestionId',
             'QuestionSource']
    with open(fname, 'r', encoding='utf-8') as fp:
      kv_items = ijson.kvitems(fp, 'Data.item')
      # print(type(kv_items))  # <class '_yajl2.kvitems'>
      getqa_res = getqa_entries(kv_items, qakeys, numqa)
      fp.close()
      return getqa_res

def print_sample_qa(qa_dict_list):
  """
  print some qa samples from given qa list
  """
  qacnt = 0
  num_qa = len(qa_dict_list)
  for i in range(num_qa):
    if (qacnt < 5) or (qacnt % 10000 == 0) or (qacnt > (num_qa-5)):
      qa_i = qa_dict_list[i]
      # print(qa_i.keys())
      # ans = qa_i['Answer']['Value']
      ans = 'TBD'
      if 'Answer' in qa_i.keys():
        ans = qa_i['Answer']['Value']
      print("qa_i ind:", i, "q_id:", qa_i['QuestionId'],
            qa_i['Question'], "...", ans)
    qacnt += 1


CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 38.4 µs


In [45]:
%%time
# ijson parse events
possible_events = ['start_map', 'end_map', 'start_array', 'end_array', 'map_key']

# keys in qa file
keys_main = ['Data', 'Domain', 'VerifiedEval', 'Version']
qakeys = ['Answer', 'EntityPages', 'Question', 'QuestionId',
             'QuestionSource']

# Try reading each file and print some Questions and Answers (Null if no Answer)
datsets_dict_list = []
for fname, scnt in fsampcnt.items():
  datsets_dict = {}
  print('---------------------------------------')
  print(f'\nNum lines in {fname}: {scnt} \n')
  qa_dict_list = getqa_entries_from_file(fname, scnt)
  qacnt = len(qa_dict_list)
  datsets_dict = {'filename': fname, 'linecnt': scnt,
                  'qacnt': qacnt, 'qa_dict_list': qa_dict_list}
  datsets_dict_list.append(datsets_dict)
  # print(type(qa_dict_list), qacnt)
  qacnt = print_sample_qa(qa_dict_list)
  print(f'\nNum qa in {fname}: {len(qa_dict_list)}\n')
print('---------------------------------------')

---------------------------------------

Num lines in ./data_local/triviaqa-rc/qa/wikipedia-dev.json: 450006 

qa_i ind: 0 q_id: tc_33 Which Lloyd Webber musical premiered in the US on 10th December 1993? ... Sunset Boulevard
qa_i ind: 1 q_id: tc_40 Who was the next British Prime Minister after Arthur Balfour? ... Campbell-Bannerman
qa_i ind: 2 q_id: tc_49 Who had a 70s No 1 hit with Kiss You All Over? ... Exile
qa_i ind: 3 q_id: tc_56 What claimed the life of singer Kathleen Ferrier? ... Cancer
qa_i ind: 4 q_id: tc_106 Which actress was voted Miss Greenwich Village in 1942? ... Lauren Bacall
qa_i ind: 7989 q_id: qg_4604 With a motto of Always Ready, Always There, what US military branch had it's founding on Dec 14, 1636? ... National Guard
qa_i ind: 7990 q_id: qg_4615 Who tried to steal Christmas from the town of Whoville? ... The Grinch
qa_i ind: 7991 q_id: qg_4628 What is the name of the parson mentioned in the lyrics of the Christmas carol "Winter Wonderland"? ... Brown
qa_i ind: 7

In [46]:
import textwrap
import pprint
# pp = pprint.PrettyPrinter(indent=1)
pp = pprint.PrettyPrinter(width=101, compact=True)
# kvitems(source, prefix, map_type=None, buf_size=65536, **config)
col_black = "\033[1;30m" ; col_red = "\033[1;31m" ; col_green = "\033[1;32m"
col_brown = "\033[1;33m" ; col_blue = "\033[1;34m" ; col_mag = "\033[1;35m"

In [47]:
# Collected data
num_files = len(datsets_dict_list)
for i in range(num_files):
  di = datsets_dict_list[i]
  print(f"{di['filename']:64s} linecnt: {di['linecnt']:7d}  qacnt: {di['qacnt']:6d}")

# Check a couple of getqa_entries for structure layout
def expand_entries(listind):
  fnameind = datsets_dict_list[listind]['filename']
  qacntind = datsets_dict_list[listind]['qacnt']
  qa_dict_list0 = datsets_dict_list[listind]['qa_dict_list']
  print("Num qa in dataset:", len(qa_dict_list0))
  for k in [0, 1, qacntind//4, qacntind//2, -1]:
    print(f"\n---{fnameind} qa{k} ---\n")
    pp.pprint(qa_dict_list0[k])
    # print(textwrap.fill(astr, 90))
  return

./data_local/triviaqa-rc/qa/wikipedia-dev.json                   linecnt:  450006  qacnt:   7993
./data_local/triviaqa-rc/qa/verified-wikipedia-dev.json          linecnt:   20553  qacnt:    318
./data_local/triviaqa-rc/qa/web-test-without-answers.json        linecnt:  513040  qacnt:   7607
./data_local/triviaqa-rc/qa/wikipedia-test-without-answers.json  linecnt:  124646  qacnt:   6160
./data_local/triviaqa-rc/qa/verified-web-dev.json                linecnt:   26196  qacnt:    407
./data_local/triviaqa-rc/qa/wikipedia-train.json                 linecnt: 3445860  qacnt:  61888
./data_local/triviaqa-rc/qa/web-train.json                       linecnt: 7118751  qacnt:  76496
./data_local/triviaqa-rc/qa/web-dev.json                         linecnt:  928803  qacnt:   9951


In [48]:
# Example entries ; note that some of the test files do not have the answer
for i in range(num_files):
  print('='*80)
  expand_entries(i)
print('='*80)

Num qa in dataset: 7993

---./data_local/triviaqa-rc/qa/wikipedia-dev.json qa0 ---

{'Answer': {'Aliases': ['Sunset Blvd', 'West Sunset Boulevard', 'Sunset Boulevard',
                        'Sunset Bulevard', 'Sunset Blvd.'],
            'MatchedWikiEntityName': 'Sunset Boulevard',
            'NormalizedAliases': ['sunset boulevard', 'sunset bulevard', 'west sunset boulevard',
                                  'sunset blvd'],
            'NormalizedMatchedWikiEntityName': 'sunset boulevard',
            'NormalizedValue': 'sunset boulevard',
            'Type': 'WikipediaEntity',
            'Value': 'Sunset Boulevard'},
 'EntityPages': [{'DocSource': 'TagMe',
                  'Filename': 'Andrew_Lloyd_Webber.txt',
                  'Title': 'Andrew Lloyd Webber'}],
 'Question': 'Which Lloyd Webber musical premiered in the US on 10th December 1993?',
 'QuestionId': 'tc_33',
 'QuestionSource': 'http://www.triviacountry.com/'}

---./data_local/triviaqa-rc/qa/wikipedia-dev.json qa1 --

## Summary - Data Exploration of triviaqa dataset  

The dataset used here was downloaded to a local GDrive from the [triviaqa github reference](https://github.com/mandarjoshi90/triviaqa) for the paper "TriviaQA: A Large Scale Distantly Supervised Challenge Dataset for Reading Comprehension"  This dataset is ready for use on triviqa models discussed in the paper.

Here, used ijson, instead of the utils in the paper to walk thru the files and understand how the data is organizaed. The data is from the Wikipedia pages and Web pages from 2017, when the paper was written. There are   
*   wikipedia-train and web-train data for training
*   wikipedia-dev and web-dev data for validation
*   verified-wikipedia-dev and verified-web-dev data for evaluation
*   wikipedia-test-without-answers and web-test-without-answers data for test


The following list of files are present in the triviaqa dataset.  </br>
*   ./data_local/triviaqa-rc/qa/wikipedia-dev.json                   linecnt:  450006  qacnt:   6660  </br>
*   ./data_local/triviaqa-rc/qa/verified-wikipedia-dev.json          linecnt:   20553  qacnt:    265  </br>
*   ./data_local/triviaqa-rc/qa/web-test-without-answers.json        linecnt:  513040  qacnt:   6339  </br>
*   ./data_local/triviaqa-rc/qa/wikipedia-test-without-answers.json  linecnt:  124646  qacnt:   5134  </br>
*   ./data_local/triviaqa-rc/qa/verified-web-dev.json                linecnt:   26196  qacnt:    339  </br>
*   ./data_local/triviaqa-rc/qa/wikipedia-train.json                 linecnt: 3445860  qacnt:  51573  </br>
*   ./data_local/triviaqa-rc/qa/web-train.json                       linecnt: 7118751  qacnt:  63746  </br>
*   ./data_local/triviaqa-rc/qa/web-dev.json                         linecnt:  928803  qacnt:   8292  </br>

A few sample entries have been expanded and printed. Each question, answer and related info is held in a dictionary, with the following keys.</br>

qakeys = ['Answer', 'EntityPages', 'Question', 'QuestionId',
             'QuestionSource']  

 Here the Entity Pages hold the text that lead to the answer and are files held in the local drive. these files were created in the original dataset from Wiki or Web Pages.

This data can now be used to try more recent NLP models. We may be able to use just the questions and answers and not use the Entity Pages. What current NLP/LLM to use and how to leverage the triviaqa dataset is to be explored while proceeding further with the project.

ijson-Ref: </br>
https://pypi.org/project/ijson/   </br>
https://www.kaggle.com/code/xxxxyyyy80008/how-to-read-a-big-json-file-with-python/notebook  </br>
https://stackoverflow.com/questions/69738173/ijson-kvitems-unexpected-behaviour   </br>
https://www.aylakhan.tech/?p=27    </br>